In [7]:
import os
import sys
sys.path.append('/Automatic-Circuit-Discovery/')
sys.path.append('..')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor
import numpy as np
import matplotlib.pyplot as plt
import json

from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.docstring.utils import get_all_docstring_things
from utils.prune_utils import get_3_caches, split_layers_and_heads
from ACDCPPExperiment import ACDCPPExperiment
device = t.device("cuda" if t.cuda.is_available() else "cpu")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu


/var/folders/tn/nyh0947d6sngjfvtp7hbb0jr0000gn/T/ipykernel_76242/1557166804.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/var/folders/tn/nyh0947d6sngjfvtp7hbb0jr0000gn/T/ipykernel_76242/1557166804.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')


In [8]:
all_docstring_items = get_all_docstring_things(num_examples=40, seq_len=5, device=device, metric_name='docstring_metric', correct_incorrect_wandb=False)

tl_model = all_docstring_items.tl_model
validation_metric = all_docstring_items.validation_metric
validation_data = all_docstring_items.validation_data
validation_labels = all_docstring_items.validation_labels
validation_patch_data = all_docstring_items.validation_patch_data
test_metrics = all_docstring_items.test_metrics
test_data = all_docstring_items.test_data
test_labels = all_docstring_items.test_labels
test_patch_data = all_docstring_items.test_patch_data

Loaded pretrained model attn-only-4l into HookedTransformer
Moving model to device:  cpu


In [9]:
def abs_docstring_metric(logits):
    return -abs(test_metrics['docstring_metric'](logits))

# ACDC++ run

In [10]:
RUN_NAME = "docstring_abs_lin_num_passes"

ACDCPP_THRESHOLDS = np.array([0.00000000e+00, 2.38284811e-05, 6.82082755e-05, 1.50934298e-04,
       2.18203088e-04, 3.29843082e-04, 4.79909359e-04, 6.15596189e-04,
       7.34498259e-04, 9.68689215e-04, 1.22944312e-03, 1.50710775e-03,
       1.85378385e-03, 2.22017476e-03, 2.56084721e-03, 2.89705396e-03,
       3.27157741e-03, 3.67312180e-03, 4.13499493e-03, 4.58357995e-03,
       5.22358762e-03, 5.91202918e-03, 6.79141749e-03, 7.49874394e-03,
       8.78367666e-03, 1.01053845e-02, 1.13945547e-02, 1.32022006e-02,
       1.55601408e-02, 1.83076523e-02, 2.12819129e-02, 2.54760347e-02,
       3.21723744e-02, 4.07917686e-02, 5.32582477e-02, 6.67442381e-02,
       1.04569145e-01, 2.13512376e-01, 7.12125480e-01, 4.55413389e+00])
ACDC_THRESHOLDS = 0.08 * np.ones(len(ACDCPP_THRESHOLDS))

print(f"{ACDCPP_THRESHOLDS=}")
print(f"{ACDC_THRESHOLDS=}")

ACDCPP_THRESHOLDS=array([0.00000000e+00, 2.38284811e-05, 6.82082755e-05, 1.50934298e-04,
       2.18203088e-04, 3.29843082e-04, 4.79909359e-04, 6.15596189e-04,
       7.34498259e-04, 9.68689215e-04, 1.22944312e-03, 1.50710775e-03,
       1.85378385e-03, 2.22017476e-03, 2.56084721e-03, 2.89705396e-03,
       3.27157741e-03, 3.67312180e-03, 4.13499493e-03, 4.58357995e-03,
       5.22358762e-03, 5.91202918e-03, 6.79141749e-03, 7.49874394e-03,
       8.78367666e-03, 1.01053845e-02, 1.13945547e-02, 1.32022006e-02,
       1.55601408e-02, 1.83076523e-02, 2.12819129e-02, 2.54760347e-02,
       3.21723744e-02, 4.07917686e-02, 5.32582477e-02, 6.67442381e-02,
       1.04569145e-01, 2.13512376e-01, 7.12125480e-01, 4.55413389e+00])
ACDC_THRESHOLDS=array([0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08,
       0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08,
       0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08,
       0.08, 0.08, 0.08, 0.08, 0.08,

In [ ]:
combined_exp = ACDCPPExperiment(
    tl_model,
    test_data,
    test_patch_data,
    acdc_metric=test_metrics['docstring_metric'],
    acdcpp_metric=abs_docstring_metric,
    acdc_thresholds=ACDC_THRESHOLDS,
    acdcpp_thresholds=ACDCPP_THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=0,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    thresholds_combined=False,
)
present_edge_attrs, num_passes, acdcpp_attrs = combined_exp.run()